In [4]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas_datareader-0.10.0-py3-none-any.whl (109 kB)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Imports

In [18]:
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import yfinance as yf

Start and End Dates

In [12]:
start = datetime.datetime(2020, 1, 1) # format in yyyy, mm, dd
end = datetime.datetime(2023, 8, 20)

# Code

Function to get stock data

In [33]:
stocks = ["MRNA", "PFE", "JNJ", "GOOGL", 
          "META", "AAPL", "COST", "WMT", "KR", "JPM", 
          "BAC", "HSBC"]
portfolio = yf.download(stocks, start="2020-01-01", end="2023-08-21")['Adj Close']

[*********************100%%**********************]  12 of 12 completed


In [34]:
portfolio

,AAPL,BAC,COST,GOOGL,HSBC,JNJ,JPM,KR,META,MRNA,PFE,WMT
Date,,,,,,,,,,,,
2020-01-02,73.249031,32.802502,276.607544,68.433998,33.379887,133.096054,125.942886,26.513744,209.779999,19.230000,32.239510,112.085144
2020-01-03,72.536888,32.121410,276.835297,68.075996,32.862690,131.555115,124.280876,26.550753,208.669998,18.889999,32.066536,111.095650
2020-01-06,73.114883,32.075401,276.911255,69.890503,32.752472,131.390976,124.182060,26.643263,212.600006,18.129999,32.025356,110.869484
2020-01-07,72.771027,31.863707,276.474731,69.755501,32.506599,132.193359,122.070930,26.448990,213.059998,17.780001,31.918266,109.842308
2020-01-08,73.941635,32.185856,279.644165,70.251999,32.430286,132.175125,123.023163,26.291719,215.220001,17.980000,32.173615,109.465363
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-14,179.460007,30.930000,561.469971,131.330002,39.779999,173.440002,154.770004,48.560001,306.190002,100.000000,36.070000,160.000000
2023-08-15,177.449997,29.940001,558.510010,129.779999,38.619999,172.940002,150.830002,47.950001,301.950012,96.410004,35.389999,159.179993
2023-08-16,176.570007,29.290001,559.049988,128.699997,37.990002,172.389999,150.190002,47.020000,294.290009,98.919998,35.480000,159.259995


In [35]:
portfolio.to_csv("portfolio.csv", index=False)

In [37]:
portfolio = pd.read_csv("portfolio.csv")

Mean variance optimization (modern portfolio theory) doesn't work very well. The goal of MV optimization is to find portfolios that optimally diversify risk without reducing expected return and to facilitate portfolio construction. Basically for a given level of risk, get the highest possible return. Mean variance optimization doesn’t perform very well since it makes many simplifying assumptions, such as returns being normally distributed and the need for an invertible covariance matrix. Fortunately, methods like HRP and mCVAR address these limitations. 

Come back to this

# MVO - Mean Variance Optimization

In [45]:
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.3.0-cp39-cp39-macosx_10_9_x86_64.whl (10.2 MB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [43]:
from pypfopt import HRPOpt

In [46]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage


mu = mean_historical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()

In [49]:
S

,AAPL,BAC,COST,GOOGL,HSBC,JNJ,JPM,KR,META,MRNA,PFE,WMT
AAPL,0.121165,0.061165,0.053809,0.080949,0.034183,0.031554,0.056008,0.018102,0.099681,0.046887,0.031710,0.034055
BAC,0.061165,0.154627,0.032500,0.060549,0.079432,0.035268,0.126314,0.013358,0.064866,0.008361,0.039712,0.026071
COST,0.053809,0.032500,0.068913,0.045715,0.017938,0.024322,0.029896,0.028656,0.050831,0.029599,0.024225,0.038938
GOOGL,0.080949,0.060549,0.045715,0.117288,0.034105,0.027032,0.053859,0.011362,0.110004,0.041717,0.030035,0.027376
HSBC,0.034183,0.079432,0.017938,0.034105,0.104468,0.020430,0.071834,0.006702,0.035027,0.000740,0.021567,0.012730
JNJ,0.031554,0.035268,0.024322,0.027032,0.020430,0.047214,0.032547,0.016159,0.025841,0.016916,0.033679,0.022980
JPM,0.056008,0.126314,0.029896,0.053859,0.071834,0.032547,0.128703,0.009901,0.056887,0.005120,0.037643,0.022814
KR,0.018102,0.013358,0.028656,0.011362,0.006702,0.016159,0.009901,0.094369,0.011114,0.010723,0.014678,0.030302
META,0.099681,0.064866,0.050831,0.110004,0.035027,0.025841,0.056887,0.011114,0.231565,0.058201,0.029355,0.027395
MRNA,0.046887,0.008361,0.029599,0.041717,0.000740,0.016916,0.005120,0.010723,0.058201,0.627032,0.045920,0.020167


In [50]:
mu

AAPL     0.270715
BAC     -0.032425
COST     0.205818
GOOGL    0.187276
HSBC     0.033860
JNJ      0.074184
JPM      0.047438
KR       0.175085
META     0.086410
MRNA     0.583279
PFE      0.036102
WMT      0.099266
dtype: float64

In [58]:
from pypfopt.efficient_frontier import EfficientFrontier

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
dict(cleaned_weights)

{'AAPL': 0.29354,
 'BAC': 0.0,
 'COST': 0.26256,
 'GOOGL': 0.0,
 'HSBC': 0.0,
 'JNJ': 0.0,
 'JPM': 0.0,
 'KR': 0.25644,
 'META': 0.0,
 'MRNA': 0.18746,
 'PFE': 0.0,
 'WMT': 0.0}

In [59]:
ef.portfolio_performance(verbose=True)

Expected annual return: 28.8%
Annual volatility: 26.0%
Sharpe Ratio: 1.03


(0.2877439364785551, 0.25964182826045495, 1.0312049421019045)

In [62]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
latest_prices = get_latest_prices(portfolio)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da.greedy_portfolio()
"Discrete allocation:", allocation

('Discrete allocation:', {'AAPL': 168, 'COST': 48, 'KR': 539, 'MRNA': 185})

In [63]:
"Funds remaining: ${:.2f}".format(leftover)

'Funds remaining: $88.63'

# HRP - Hierarchical Risk Parity

In [64]:
returns = portfolio.pct_change().dropna()

In [65]:
returns

,AAPL,BAC,COST,GOOGL,HSBC,JNJ,JPM,KR,META,MRNA,PFE,WMT
1,-0.009722,-0.020763,0.000823,-0.005231,-0.015494,-0.011578,-0.013197,0.001396,-0.005291,-0.017681,-0.005365,-0.008828
2,0.007968,-0.001432,0.000274,0.026654,-0.003354,-0.001248,-0.000795,0.003484,0.018834,-0.040233,-0.001284,-0.002036
3,-0.004703,-0.006600,-0.001576,-0.001932,-0.007507,0.006107,-0.017000,-0.007292,0.002164,-0.019305,-0.003344,-0.009265
4,0.016086,0.010110,0.011464,0.007118,-0.002348,-0.000138,0.007801,-0.005946,0.010138,0.011249,0.008000,-0.003432
5,0.021241,0.001715,0.016051,0.010498,0.006536,0.002966,0.003651,0.000704,0.014311,0.023359,-0.004352,0.010331
...,...,...,...,...,...,...,...,...,...,...,...,...
909,0.009393,-0.011505,-0.002647,0.013662,-0.008227,-0.002358,0.002072,-0.002670,0.015084,-0.014681,0.000832,-0.007444
910,-0.011200,-0.032008,-0.005272,-0.011802,-0.029160,-0.002883,-0.025457,-0.012562,-0.013848,-0.035900,-0.018852,-0.005125
911,-0.004959,-0.021710,0.000967,-0.008322,-0.016313,-0.003180,-0.004243,-0.019395,-0.025368,0.026035,0.002543,0.000503
912,-0.014555,-0.000341,-0.021358,0.009479,-0.001579,0.009397,-0.010387,-0.005955,-0.031262,0.073999,0.029030,-0.022416


In [67]:
hrp = HRPOpt(returns)
hrp_weights = hrp.optimize()
dict(hrp_weights)

{'AAPL': 0.07200234851064752,
 'BAC': 0.050360057528187466,
 'COST': 0.0857983868834488,
 'GOOGL': 0.04927770673653624,
 'HSBC': 0.12312646821855108,
 'JNJ': 0.1423463005485768,
 'JPM': 0.04597943836034879,
 'KR': 0.13683066529504848,
 'META': 0.02522908532070127,
 'MRNA': 0.021308267690548353,
 'PFE': 0.07799326149024648,
 'WMT': 0.16974801341715878}

In [68]:
hrp.portfolio_performance(verbose=True)

Expected annual return: 16.0%
Annual volatility: 18.7%
Sharpe Ratio: 0.75


(0.15985368772332914, 0.18735101942554594, 0.7464794595308171)

In [69]:
da_hrp = DiscreteAllocation(hrp_weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da_hrp.greedy_portfolio()
"Discrete allocation (HRP):", allocation

('Discrete allocation (HRP):',
 {'WMT': 107,
  'JNJ': 82,
  'KR': 287,
  'HSBC': 327,
  'COST': 16,
  'PFE': 212,
  'AAPL': 41,
  'BAC': 173,
  'GOOGL': 39,
  'JPM': 31,
  'META': 9,
  'MRNA': 21})

In [70]:
"Funds remaining (HRP): ${:.2f}".format(leftover)

'Funds remaining (HRP): $36.54'